# BIN to TIF Conversion (Longitude, Latitude, Value)

This notebook demonstrates how to convert three raw **binary files (.bin)** containing 
**longitude, latitude, and values** into a properly georeferenced **TIF** raster.  

It uses `rasterio` to define the grid resolution and export the result.  


## Import needed libraries

In [1]:

import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path


## Parameters

In [31]:

# Input directory containing BIN files
input_dir = Path("data/bin_files")

# Output directory for TIF files
output_dir = Path("data/tif_files")
output_dir.mkdir(parents=True, exist_ok=True)

# Resolution in degrees (pixel size)
res_deg = 1 / 64   # e.g.: ~0.0156°

# File names 
lat_file = input_dir / "latitude.bin"
lon_file = input_dir / "longitude.bin"
val_file = input_dir / "s0linear.bin"

# Output TIF file
output_file = output_dir / "output_image.tif"

# Nodata values
nodata_value = -1.0e31
output_nodata = -9999


## Load and filter initial data

In [ ]:

# Load binary files
lon = np.fromfile(lon_file, dtype="<f4")
lat = np.fromfile(lat_file, dtype="<f4")
val = np.fromfile(val_file, dtype="<f4")

# Apply mask to keep only valid values
mask = (
    (lon > -180) & (lon < 180) &
    (lat > -90) & (lat < 90) &
    (val != nodata_value)
)

lon = lon[mask]
lat = lat[mask]
val = val[mask]

print(f"{len(val)} valid points remain after filtering.")


## Define raster grid from resolution

In [ ]:

# Grid bounds
min_lon, max_lon = lon.min(), lon.max()
min_lat, max_lat = lat.min(), lat.max()

# Grid size in pixels
ncols = int(np.ceil((max_lon - min_lon) / res_deg))
nrows = int(np.ceil((max_lat - min_lat) / res_deg))

print(f"Grid size: {nrows} rows x {ncols} cols")

# Raster transform
transform = from_origin(min_lon, max_lat, res_deg, res_deg)


## Rasterization of values

In [41]:

# Compute row/col indices for each point
col = ((lon - min_lon) / res_deg).astype(int)
row = ((max_lat - lat) / res_deg).astype(int)

# Initialize empty grid
grid = np.full((nrows, ncols), np.nan, dtype=np.float32)

# Assign values to the grid
grid[row, col] = val


## Export

In [ ]:

with rasterio.open(
    output_tif,
    "w",
    driver="GTiff",
    height=nrows,
    width=ncols,
    count=1,
    dtype=grid.dtype,
    crs="EPSG:4326",
    transform=transform,
    nodata=output_nodata
) as dst:
    dst.write(np.nan_to_num(grid, nan=output_nodata), 1)

print(f"✅ Georeferenced TIF saved at {output_tif}")


## Validation and preview

In [ ]:

# Show min/max values
print("Longitude range:", lon.min(), "→", lon.max())
print("Latitude range:", lat.min(), "→", lat.max())
print("Value range:", val.min(), "→", val.max())

# Show raster preview
plt.imshow(grid, cmap="viridis", extent=[min_lon, max_lon, min_lat, max_lat])
plt.colorbar(label="Value")
plt.title("GeoTIFF Preview")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# Quick table preview of first points
df = pd.DataFrame({
    "Longitude": lon[:5],
    "Latitude": lat[:5],
    "Value": val[:5]
})
print(df)


---
## 📌 Notes
- Adapt the resolution `res_deg` depending on the desired pixel size.  
- The CRS is set to `EPSG:4326` (WGS84 lat/lon). Change if needed.  
- Input `.bin` files must contain floats (`dtype='<f4'`). Adapt dtype if different.  
- For very large datasets, consider using chunked processing or memory mapping.  
---
